In [1]:
#the dataset we are using contains 5000 imdb movies of different genres,time periods and plots.
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
df=pd.read_csv(r'https://query.data.world/s/qvc3lchzrpjbvcqegp3ababm7yzut5')
df
#importing the dataset

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5038,Color,Scott Smith,1.0,87.0,2.0,318.0,Daphne Zuniga,637.0,NaN,Comedy|Drama,...,6.0,English,Canada,NaN,NaN,2013.0,470.0,7.7,NaN,84
5039,Color,NaN,43.0,43.0,NaN,319.0,Valorie Curry,841.0,NaN,Crime|Drama|Mystery|Thriller,...,359.0,English,USA,TV-14,NaN,NaN,593.0,7.5,16.00,32000
5040,Color,Benjamin Roberds,13.0,76.0,0.0,0.0,Maxwell Moody,0.0,NaN,Drama|Horror|Thriller,...,3.0,English,USA,NaN,1400.0,2013.0,0.0,6.3,NaN,16
5041,Color,Daniel Hsia,14.0,100.0,0.0,489.0,Daniel Henney,946.0,10443.0,Comedy|Drama|Romance,...,9.0,English,USA,PG-13,NaN,2012.0,719.0,6.3,2.35,660


In [17]:
#dropping the columns we dont need
#dropping the movies with no plots and director's name because these,I believe are these are the two most
#important features to determine the similarity between movies
df=df.dropna(axis='rows',subset=['plot_keywords','director_name','movie_title']).reset_index(drop=True)
df=df[['director_name','actor_2_name','genres','actor_1_name','movie_title','actor_3_name','plot_keywords','movie_imdb_link','imdb_score']]

In [18]:
df

,director_name,actor_2_name,genres,actor_1_name,movie_title,actor_3_name,plot_keywords,movie_imdb_link,imdb_score
0,James Cameron,Joel David Moore,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,Wes Studi,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,7.9
1,Gore Verbinski,Orlando Bloom,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,Jack Davenport,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,7.1
2,Sam Mendes,Rory Kinnear,Action|Adventure|Thriller,Christoph Waltz,Spectre,Stephanie Sigman,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,6.8
3,Christopher Nolan,Christian Bale,Action|Thriller,Tom Hardy,The Dark Knight Rises,Joseph Gordon-Levitt,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,8.5
4,Andrew Stanton,Samantha Morton,Action|Adventure|Sci-Fi,Daryl Sabara,John Carter,Polly Walker,alien|american civil war|male nipple|mars|prin...,http://www.imdb.com/title/tt0401729/?ref_=fn_t...,6.6
...,...,...,...,...,...,...,...,...,...
4790,Robert Rodriguez,Peter Marquardt,Action|Crime|Drama|Romance|Thriller,Carlos Gallardo,El Mariachi,Consuelo Gómez,assassin|death|guitar|gun|mariachi,http://www.imdb.com/title/tt0104815/?ref_=fn_t...,6.9
4791,Anthony Vallone,John Considine,Crime|Drama,Richard Jewell,The Mongol King,Sara Stepnicka,jewell|mongol|nostradamus|stepnicka|vallone,http://www.imdb.com/title/tt0430371/?ref_=fn_t...,7.8
4792,Edward Burns,Caitlin FitzGerald,Comedy|Drama,Kerry Bishé,Newlyweds,Daniella Pineda,written and directed by cast member,http://www.imdb.com/title/tt1880418/?ref_=fn_t...,6.4
4793,Scott Smith,Daphne Zuniga,Comedy|Drama,Eric Mabius,Signed Sealed Delivered,Crystal Lowe,fraud|postal worker|prison|theft|trial,http://www.imdb.com/title/tt3000844/?ref_=fn_t...,7.7


In [19]:
#merging the actor names into one and dropping the rest
df['actors']=df['actor_1_name']+'|'+df['actor_2_name']+'|'+df['actor_3_name']


In [20]:
df=df.drop(['actor_1_name','actor_2_name','actor_3_name'],axis=1)
df

,director_name,genres,movie_title,plot_keywords,movie_imdb_link,imdb_score,actors
0,James Cameron,Action|Adventure|Fantasy|Sci-Fi,Avatar,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,7.9,CCH Pounder|Joel David Moore|Wes Studi
1,Gore Verbinski,Action|Adventure|Fantasy,Pirates of the Caribbean: At World's End,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,7.1,Johnny Depp|Orlando Bloom|Jack Davenport
2,Sam Mendes,Action|Adventure|Thriller,Spectre,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,6.8,Christoph Waltz|Rory Kinnear|Stephanie Sigman
3,Christopher Nolan,Action|Thriller,The Dark Knight Rises,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,8.5,Tom Hardy|Christian Bale|Joseph Gordon-Levitt
4,Andrew Stanton,Action|Adventure|Sci-Fi,John Carter,alien|american civil war|male nipple|mars|prin...,http://www.imdb.com/title/tt0401729/?ref_=fn_t...,6.6,Daryl Sabara|Samantha Morton|Polly Walker
...,...,...,...,...,...,...,...
4790,Robert Rodriguez,Action|Crime|Drama|Romance|Thriller,El Mariachi,assassin|death|guitar|gun|mariachi,http://www.imdb.com/title/tt0104815/?ref_=fn_t...,6.9,Carlos Gallardo|Peter Marquardt|Consuelo Gómez
4791,Anthony Vallone,Crime|Drama,The Mongol King,jewell|mongol|nostradamus|stepnicka|vallone,http://www.imdb.com/title/tt0430371/?ref_=fn_t...,7.8,Richard Jewell|John Considine|Sara Stepnicka
4792,Edward Burns,Comedy|Drama,Newlyweds,written and directed by cast member,http://www.imdb.com/title/tt1880418/?ref_=fn_t...,6.4,Kerry Bishé|Caitlin FitzGerald|Daniella Pineda
4793,Scott Smith,Comedy|Drama,Signed Sealed Delivered,fraud|postal worker|prison|theft|trial,http://www.imdb.com/title/tt3000844/?ref_=fn_t...,7.7,Eric Mabius|Daphne Zuniga|Crystal Lowe


In [21]:
#changing the order of the columns
#this step is not necessary,I just did it because I liked it in this order 
order=[2,5,1,0,3,6,4]
df=df[df.columns[order]]
df


,movie_title,imdb_score,genres,director_name,plot_keywords,actors,movie_imdb_link
0,Avatar,7.9,Action|Adventure|Fantasy|Sci-Fi,James Cameron,avatar|future|marine|native|paraplegic,CCH Pounder|Joel David Moore|Wes Studi,http://www.imdb.com/title/tt0499549/?ref_=fn_t...
1,Pirates of the Caribbean: At World's End,7.1,Action|Adventure|Fantasy,Gore Verbinski,goddess|marriage ceremony|marriage proposal|pi...,Johnny Depp|Orlando Bloom|Jack Davenport,http://www.imdb.com/title/tt0449088/?ref_=fn_t...
2,Spectre,6.8,Action|Adventure|Thriller,Sam Mendes,bomb|espionage|sequel|spy|terrorist,Christoph Waltz|Rory Kinnear|Stephanie Sigman,http://www.imdb.com/title/tt2379713/?ref_=fn_t...
3,The Dark Knight Rises,8.5,Action|Thriller,Christopher Nolan,deception|imprisonment|lawlessness|police offi...,Tom Hardy|Christian Bale|Joseph Gordon-Levitt,http://www.imdb.com/title/tt1345836/?ref_=fn_t...
4,John Carter,6.6,Action|Adventure|Sci-Fi,Andrew Stanton,alien|american civil war|male nipple|mars|prin...,Daryl Sabara|Samantha Morton|Polly Walker,http://www.imdb.com/title/tt0401729/?ref_=fn_t...
...,...,...,...,...,...,...,...
4790,El Mariachi,6.9,Action|Crime|Drama|Romance|Thriller,Robert Rodriguez,assassin|death|guitar|gun|mariachi,Carlos Gallardo|Peter Marquardt|Consuelo Gómez,http://www.imdb.com/title/tt0104815/?ref_=fn_t...
4791,The Mongol King,7.8,Crime|Drama,Anthony Vallone,jewell|mongol|nostradamus|stepnicka|vallone,Richard Jewell|John Considine|Sara Stepnicka,http://www.imdb.com/title/tt0430371/?ref_=fn_t...
4792,Newlyweds,6.4,Comedy|Drama,Edward Burns,written and directed by cast member,Kerry Bishé|Caitlin FitzGerald|Daniella Pineda,http://www.imdb.com/title/tt1880418/?ref_=fn_t...
4793,Signed Sealed Delivered,7.7,Comedy|Drama,Scott Smith,fraud|postal worker|prison|theft|trial,Eric Mabius|Daphne Zuniga|Crystal Lowe,http://www.imdb.com/title/tt3000844/?ref_=fn_t...


In [22]:
df=df.sort_values(by='imdb_score',ascending=False).reset_index(drop=True)
df
#sorting data based on descending imdb scores,highest first


,movie_title,imdb_score,genres,director_name,plot_keywords,actors,movie_imdb_link
0,The Shawshank Redemption,9.3,Crime|Drama,Frank Darabont,escape from prison|first person narration|pris...,Morgan Freeman|Jeffrey DeMunn|Bob Gunton,http://www.imdb.com/title/tt0111161/?ref_=fn_t...
1,The Godfather,9.2,Crime|Drama,Francis Ford Coppola,crime family|mafia|organized crime|patriarch|r...,Al Pacino|Marlon Brando|Robert Duvall,http://www.imdb.com/title/tt0068646/?ref_=fn_t...
2,The Dark Knight,9.0,Action|Crime|Drama|Thriller,Christopher Nolan,based on comic book|dc comics|psychopath|star ...,Christian Bale|Heath Ledger|Morgan Freeman,http://www.imdb.com/title/tt0468569/?ref_=fn_t...
3,The Godfather: Part II,9.0,Crime|Drama,Francis Ford Coppola,1950s|corrupt politician|lake tahoe nevada|mel...,Robert De Niro|Al Pacino|Robert Duvall,http://www.imdb.com/title/tt0071562/?ref_=fn_t...
4,Schindler's List,8.9,Biography|Drama|History,Steven Spielberg,german|german soldier|jew|jewish|nazi,Liam Neeson|Embeth Davidtz|Caroline Goodall,http://www.imdb.com/title/tt0108052/?ref_=fn_t...
...,...,...,...,...,...,...,...
4790,Superbabies: Baby Geniuses 2,1.9,Comedy|Family|Sci-Fi,Bob Clark,computer|good versus evil|mind control|science...,Scott Baio|Vanessa Angel|Peter Wingfield,http://www.imdb.com/title/tt0270846/?ref_=fn_t...
4791,Disaster Movie,1.9,Comedy,Jason Friedberg,disaster|glass|mousetrap|natural disaster|scat...,Carmen Electra|Tony Cox|Ike Barinholtz,http://www.imdb.com/title/tt1213644/?ref_=fn_t...
4792,The Helix... Loaded,1.9,Action|Adventure|Comedy|Fantasy|Sci-Fi,A. Raven Cruz,cnn reporter|ellipsis in title|matrix|punctuat...,Scott Levy|Vanilla Ice|Jennifer Sky,http://www.imdb.com/title/tt0401462/?ref_=fn_t...
4793,Foodfight!,1.7,Action|Animation|Comedy|Family|Fantasy,Lawrence Kasanoff,femme fatale|food|pun|supermarket|troubled pro...,Jerry Stiller|Larry Miller|Harvey Fierstein,http://www.imdb.com/title/tt0249516/?ref_=fn_t...


In [23]:
#nobody wants low rated imdb movies so removing movies with imdb<6
df=df.drop(df[df['imdb_score']<6].index)
df
#we started with 5000 movies and now we are left with 3413



,movie_title,imdb_score,genres,director_name,plot_keywords,actors,movie_imdb_link
0,The Shawshank Redemption,9.3,Crime|Drama,Frank Darabont,escape from prison|first person narration|pris...,Morgan Freeman|Jeffrey DeMunn|Bob Gunton,http://www.imdb.com/title/tt0111161/?ref_=fn_t...
1,The Godfather,9.2,Crime|Drama,Francis Ford Coppola,crime family|mafia|organized crime|patriarch|r...,Al Pacino|Marlon Brando|Robert Duvall,http://www.imdb.com/title/tt0068646/?ref_=fn_t...
2,The Dark Knight,9.0,Action|Crime|Drama|Thriller,Christopher Nolan,based on comic book|dc comics|psychopath|star ...,Christian Bale|Heath Ledger|Morgan Freeman,http://www.imdb.com/title/tt0468569/?ref_=fn_t...
3,The Godfather: Part II,9.0,Crime|Drama,Francis Ford Coppola,1950s|corrupt politician|lake tahoe nevada|mel...,Robert De Niro|Al Pacino|Robert Duvall,http://www.imdb.com/title/tt0071562/?ref_=fn_t...
4,Schindler's List,8.9,Biography|Drama|History,Steven Spielberg,german|german soldier|jew|jewish|nazi,Liam Neeson|Embeth Davidtz|Caroline Goodall,http://www.imdb.com/title/tt0108052/?ref_=fn_t...
...,...,...,...,...,...,...,...
3408,The Hotel New Hampshire,6.0,Comedy|Drama|Romance,Tony Richardson,football|hotel|teenage girl|terrorist|wien,Joely Richardson|Beau Bridges|Nastassja Kinski,http://www.imdb.com/title/tt0087428/?ref_=fn_t...
3409,Raising Helen,6.0,Comedy|Drama|Romance,Garry Marshall,fashion show|female protagonist|love|pastor|qu...,Amber Valletta|Felicity Huffman|Spencer Breslin,http://www.imdb.com/title/tt0350028/?ref_=fn_t...
3410,Roll Bounce,6.0,Comedy|Drama|Music|Romance,Malcolm D. Lee,1970s|boy|competition|roller skates|tubesocks,Jurnee Smollett-Bell|Brandon T. Jackson|Shad Moss,http://www.imdb.com/title/tt0403455/?ref_=fn_t...
3411,For Colored Girls,6.0,Drama,Tyler Perry,african american protagonist|female protagonis...,Omari Hardwick|Loretta Devine|Kimberly Elise,http://www.imdb.com/title/tt1405500/?ref_=fn_t...


In [121]:
#removing spaces from actor and directors names to avoid misconception
#suppose we have Robert Duvall and Robert De Niro 
#Since their first names match our model will consider a similarity even if there isn't
df['director_name']=df['director_name'].str.replace(" ","")
df['actors']=df['actors'].str.replace(" ","")
df['movie_titles']=df['movie_title'].str.replace("\xa0","")
df
#now unless and untill the full name doesn't match our model will not consider them same

,movie_title,imdb_score,genres,director_name,plot_keywords,actors,movie_imdb_link,movie_titles
0,The Shawshank Redemption,9.3,Crime|Drama,FrankDarabont,escape from prison|first person narration|pris...,MorganFreeman|JeffreyDeMunn|BobGunton,http://www.imdb.com/title/tt0111161/?ref_=fn_t...,The Shawshank Redemption
1,The Godfather,9.2,Crime|Drama,FrancisFordCoppola,crime family|mafia|organized crime|patriarch|r...,AlPacino|MarlonBrando|RobertDuvall,http://www.imdb.com/title/tt0068646/?ref_=fn_t...,The Godfather
2,The Dark Knight,9.0,Action|Crime|Drama|Thriller,ChristopherNolan,based on comic book|dc comics|psychopath|star ...,ChristianBale|HeathLedger|MorganFreeman,http://www.imdb.com/title/tt0468569/?ref_=fn_t...,The Dark Knight
3,The Godfather: Part II,9.0,Crime|Drama,FrancisFordCoppola,1950s|corrupt politician|lake tahoe nevada|mel...,RobertDeNiro|AlPacino|RobertDuvall,http://www.imdb.com/title/tt0071562/?ref_=fn_t...,The Godfather: Part II
4,Schindler's List,8.9,Biography|Drama|History,StevenSpielberg,german|german soldier|jew|jewish|nazi,LiamNeeson|EmbethDavidtz|CarolineGoodall,http://www.imdb.com/title/tt0108052/?ref_=fn_t...,Schindler's List
...,...,...,...,...,...,...,...,...
3408,The Hotel New Hampshire,6.0,Comedy|Drama|Romance,TonyRichardson,football|hotel|teenage girl|terrorist|wien,JoelyRichardson|BeauBridges|NastassjaKinski,http://www.imdb.com/title/tt0087428/?ref_=fn_t...,The Hotel New Hampshire
3409,Raising Helen,6.0,Comedy|Drama|Romance,GarryMarshall,fashion show|female protagonist|love|pastor|qu...,AmberValletta|FelicityHuffman|SpencerBreslin,http://www.imdb.com/title/tt0350028/?ref_=fn_t...,Raising Helen
3410,Roll Bounce,6.0,Comedy|Drama|Music|Romance,MalcolmD.Lee,1970s|boy|competition|roller skates|tubesocks,JurneeSmollett-Bell|BrandonT.Jackson|ShadMoss,http://www.imdb.com/title/tt0403455/?ref_=fn_t...,Roll Bounce
3411,For Colored Girls,6.0,Drama,TylerPerry,african american protagonist|female protagonis...,OmariHardwick|LorettaDevine|KimberlyElise,http://www.imdb.com/title/tt1405500/?ref_=fn_t...,For Colored Girls


In [33]:

Title=df['movie_title']
finaldata=pd.DataFrame(Title)
finaldata['Title']=finaldata['movie_title']
finaldata=finaldata.drop('movie_title',axis=1)
#bag of words
finaldata['bag_of_words']=df['actors']+' '+df['plot_keywords'] +' '+df['genres']+' '+df['director_name']
#Converting the text to lowercase to avoid duplication and removing '|'
finaldata['bag_of_words']=finaldata['bag_of_words'].str.lower()
finaldata['bag_of_words']=finaldata['bag_of_words'].str.replace("|"," ")
#setting the title column as the index 
#finaldata.set_index('Title',inplace=True,drop=True)
finaldata=finaldata.fillna(' ')
finaldata
#now our dataset is ready for modelLing


,Title,bag_of_words
0,The Shawshank Redemption,morganfreeman jeffreydemunn bobgunton escape f...
1,The Godfather,alpacino marlonbrando robertduvall crime famil...
2,The Dark Knight,christianbale heathledger morganfreeman based ...
3,The Godfather: Part II,robertdeniro alpacino robertduvall 1950s corru...
4,Schindler's List,liamneeson embethdavidtz carolinegoodall germa...
...,...,...
3408,The Hotel New Hampshire,joelyrichardson beaubridges nastassjakinski fo...
3409,Raising Helen,ambervalletta felicityhuffman spencerbreslin f...
3410,Roll Bounce,jurneesmollett-bell brandont.jackson shadmoss ...
3411,For Colored Girls,omarihardwick lorettadevine kimberlyelise afri...


In [26]:
#generating the count matrix
count=CountVectorizer()
count_matrix=count.fit_transform(finaldata['bag_of_words'])

#making a cosine similarity matrix
cosine_matrix=cosine_similarity(count_matrix,count_matrix)
cosine_matrix

array([[1.        , 0.17817416, 0.13363062, ..., 0.04950738, 0.04351941,
        0.        ],
       [0.17817416, 1.        , 0.19047619, ..., 0.05292561, 0.04652421,
        0.        ],
       [0.13363062, 0.19047619, 1.        , ..., 0.05292561, 0.04652421,
        0.04761905],
       ...,
       [0.04950738, 0.05292561, 0.05292561, ..., 1.        , 0.05170877,
        0.05292561],
       [0.04351941, 0.04652421, 0.04652421, ..., 0.05170877, 1.        ,
        0.        ],
       [0.        , 0.        , 0.04761905, ..., 0.05292561, 0.        ,
        1.        ]])

In [133]:
#first i will create a pandas series of all the movie titles to access the
#movies using indices of the series in the function

indices=pd.Series(finaldata['Title'])
indices=indices.str.replace("\xa0","")
#now we define the function which takes the movie title as input
        
def recommend(title,cosine_matrix=cosine_matrix):
    #initializing empty dictionary
    recommendations=defaultdict(list)
    #getting the index of the movie that matches the title
    idx=0
    while True:
        if indices[idx]==title:
            break
        else:
            idx+=1
    #idx=indices[indices==title].index[0]
    #creating a series with the similarity scores
    sim_series=pd.Series(cosine_matrix[idx]).sort_values(ascending=False)
    #slicing the top 10 movies 
    top_10_index=list(sim_series.iloc[1:11].index)
    
    #The function will return the important data
    important_features=['movie_title','imdb_score','director_name','genres','plot_keywords','actors','movie_imdb_link']
    for i in top_10_index:
        for feature in important_features:
       
          recommendations[feature].append(df.xs(i)[feature])
        rec=pd.DataFrame(recommendations)   
    return rec 

#for dictionary return delete the last 2 lines and use -->return recommendations
  
        
    
    
    

In [134]:
recommend('The Godfather')

,movie_title,imdb_score,director_name,genres,plot_keywords,actors,movie_imdb_link
0,The French Connection,7.8,WilliamFriedkin,Action|Crime|Drama|Thriller,crime boss|drug smuggling|heroin|mafia boss|or...,RoyScheider|FernandoRey|TonyLoBianco,http://www.imdb.com/title/tt0067116/?ref_=fn_t...
1,The French Connection,7.8,WilliamFriedkin,Action|Crime|Drama|Thriller,crime boss|drug smuggling|heroin|mafia boss|or...,RoyScheider|FernandoRey|TonyLoBianco,http://www.imdb.com/title/tt0067116/?ref_=fn_t...
2,Goodfellas,8.7,MartinScorsese,Biography|Crime|Drama,betrayal|gangster|mafia|organized crime|robbery,RobertDeNiro|MikeStarr|PaulSorvino,http://www.imdb.com/title/tt0099685/?ref_=fn_t...
3,Out of the Blue,7.3,RobertSarkies,Crime|Drama,true crime,WilliamKircher|MatthewSunderland|PaulGlover,http://www.imdb.com/title/tt0839938/?ref_=fn_t...
4,Deadline - U.S.A.,7.2,RichardBrooks,Crime|Drama|Film-Noir,editor|gangster|newspaper|organized crime|press,HumphreyBogart|JimBackus|KimHunter,http://www.imdb.com/title/tt0044533/?ref_=fn_t...
5,Defendor,6.8,PeterStebbings,Comedy|Crime|Drama,gun running|organized crime|pimp|superhero|vig...,MichaelKelly|TonyNappo|CharlotteSullivan,http://www.imdb.com/title/tt1303828/?ref_=fn_t...
6,The Godfather: Part III,7.6,FrancisFordCoppola,Crime|Drama,1970s|family relationships|mob hit|opera|repea...,AlPacino|JoeMantegna|BridgetFonda,http://www.imdb.com/title/tt0099674/?ref_=fn_t...
7,The Godfather: Part II,9.0,FrancisFordCoppola,Crime|Drama,1950s|corrupt politician|lake tahoe nevada|mel...,RobertDeNiro|AlPacino|RobertDuvall,http://www.imdb.com/title/tt0071562/?ref_=fn_t...
8,Safe Men,6.1,JohnHamburg,Comedy|Crime,gangster|organized crime|robbery|rosh hashanah...,HarveyFierstein|MichaelLerner|JoshPais,http://www.imdb.com/title/tt0120813/?ref_=fn_t...
9,Hostage,6.6,Florent-EmilioSiri,Action|Crime|Drama|Mystery|Thriller,california|hostage|hostage negotiator|organize...,JimmyBennett|BruceWillis|RansfordDoherty,http://www.imdb.com/title/tt0340163/?ref_=fn_t...
